In [1]:
import os 
import pandas as pd
from collections import Counter


path_work = "/media/concha-eloko/Linux/PPT_clean"

bacteria_data_df = pd.read_csv(f"{path_work}/results_kleborate_count.tsv", sep = "\t", header = 0)

DF_info = pd.read_csv(f"{path_work}/TropiGATv2.final_df_v2.tsv", sep = "\t" ,  header = 0)

DF_info_lvl_0 = DF_info[~DF_info["KL_type_LCA"].str.contains("\\|")]
DF_info_amb = DF_info[DF_info["KL_type_LCA"].str.contains("\\|")]

#DF_info_lvl_0_filter1 = DF_info_lvl_0.drop_duplicates(subset = ["Infected_ancestor","index","prophage_id"] , keep = "first").reset_index(drop=True)

In [3]:
import os
import random
import warnings
from collections import Counter
from multiprocessing.pool import ThreadPool
import json
import math
import logging

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import (accuracy_score, f1_score, matthews_corrcoef,
                             precision_score, recall_score, roc_auc_score)
import optuna
from sklearn.model_selection import StratifiedKFold
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import GATv2Conv, HeteroConv
from tqdm import tqdm

/media/concha-eloko/Linux/conda_envs/torch_geometric/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/media/concha-eloko/Linux/conda_envs/torch_geometric/lib/python3.11/site-packages/torch_geometric/typing.py:18: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /media/concha-eloko/Linux/conda_envs/torch_geometric/lib/python3.11/site-packages/libpyg.so: undefined symbol: _ZN3c1010Dispatcher17runRecordFunctionERN2at14RecordFunctionESt17reference_wrapperIKNS_14FunctionSchemaEENS_11DispatchKeyE
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/media/concha-eloko/Linux/conda_envs/torch_geometric/lib/python3.11/site-packages/torch_geometric/typing.py:31: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /m

In [ ]:
import TropiGAT_graph
import TropiGAT_models
import 

In [2]:
! python -m pip install optuna


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 4.0 MB/s eta 0:00:002.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 37.4 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 190.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.4/602.4 kB 91.8 MB/s eta 0:00:00


# Not sure what happends there :

In [ ]:
import os 
import pandas as pd
from collections import Counter


path_work = "/media/concha-eloko/Linux/PPT_clean"

bacteria_data_df = pd.read_csv(f"{path_work}/results_kleborate_count.tsv", sep = "\t", header = 0)

DF_info = pd.read_csv(f"{path_work}/TropiGATv2.final_df_v2.tsv", sep = "\t" ,  header = 0)

DF_info_lvl_0 = DF_info[~DF_info["KL_type_LCA"].str.contains("\\|")]
DF_info_amb = DF_info[DF_info["KL_type_LCA"].str.contains("\\|")]

#DF_info_lvl_0_filter1 = DF_info_lvl_0.drop_duplicates(subset = ["Infected_ancestor","index","prophage_id"] , keep = "first").reset_index(drop=True)

In [12]:
path_prophage = "/media/concha-eloko/Linux/prediction_depolymerase_tropism/prophage_work/prophage_prediction"
f_labels=["Prophage name","K-serotype monophyletic group","Id monophyletic group","Number of clades","Number of leafs","Number of new ancestors","Number of k-type swap","Nodes k-types","Nodes k-types all"]

df_prophages_1 = pd.read_csv(f"{path_prophage}/prophage_data.clusters_80.phageboost_70.final.tsv", sep="\t", names =f_labels) 
df_prophages_2 = pd.read_csv(f"{path_work}/prophage_data.clusters_80.phageboost_70.2504.tsv", sep="\t", names =f_labels, skiprows=1) 



In [7]:
df_prophages_1

,Prophage name,K-serotype monophyletic group,Id monophyletic group,Number of clades,Number of leafs,Number of new ancestors,Number of k-type swap,Nodes k-types,Nodes k-types all
0,GCF_020615655.1__EMCPJNLC_9,KL64,n4043,1,2,1,0,None,None
1,GCF_021213885.1__GOKIIELC_105,KL19,n28462846,4,5,4,0,KL19,"KL19,KL19,KL19,KL19"
2,GCF_003666725.1__MDOIKHEP_22,KL64,n2838,1,2,1,0,None,None
3,GCF_002261745.1__NCJKIGBL_29,KL47,n23482348,2,3,2,0,KL47,"KL47,KL47"
4,GCF_003262135.1__LHIFPDKB_94,KL64,n26072607,3,4,3,0,KL64,"KL64,KL64,KL64"
...,...,...,...,...,...,...,...,...,...
60064,GCF_900516735.1__KGBILJNM_66,KL54,GCF_900516735.1,0,1,0,0,None,None
60065,GCF_020479585.1__CKJNGDMH_2,KL49,n54935493,1,2,1,0,None,None
60066,GCF_900510465.1__KNLMOLHD_60,KL14,n4610,2,3,1,0,None,None
60067,GCF_902162525.1__BAKPCPAK_6,KL164,GCF_902162525.1,0,1,0,0,None,None


In [17]:
df_prophages_2_amb = df_prophages_2[df_prophages_2["K-serotype monophyletic group"].str.contains("\\|")]
df_prophages_2_amb

,Prophage name,K-serotype monophyletic group,Id monophyletic group,Number of clades,Number of leafs,Number of new ancestors,Number of k-type swap,Nodes k-types,Nodes k-types all
167,GCF_014288695.1__phage4.fasta,KL122|KL25|KL3|KL125|KL24|KL2|KL31|KL30|KL35|K...,n5891,1,2,1,0,None,None
170,GCF_019930245.1__phage16.fasta,KL60|KL110|KL142,n43774377,5,6,1,0,None,None
250,GCF_019038845.1__phage7.fasta,KL122|KL128|KL25|KL43|KL61|KL3|KL151|KL53|KL10...,n58915891,3,4,1,0,None,None
255,GCF_014286295.1__phage3.fasta,KL122|KL43|KL25|KL61|KL3|KL151|KL48|KL53|KL125...,n54705470,1,2,1,0,None,None
259,GCF_003692695.1__phage23.fasta,KL122|KL43|KL25|KL3|KL125|KL24|KL2|KL31|KL140|...,n2021,3,4,1,0,None,None
...,...,...,...,...,...,...,...,...,...
75565,GCF_900507595.1__phage7.fasta,KL15|KL30,49.5/100,34,35,1,0,None,None
76235,GCF_016635675.1__phage0.fasta,KL30|KL133|KL11,n5931,13,14,3,1,"KL30|KL64|KL14|KL133|KL10|KL47|KL11|KL39,KL47","KL30|KL64|KL14|KL133|KL10|KL47|KL11|KL39,KL11|..."
76249,GCF_900507185.1__phage10.fasta,KL28|KL61|KL110,n13911391,3,4,1,0,None,None
76529,GCF_020178015.1__phage0.fasta,KL43|KL25|KL3|KL136|KL125|KL24|KL116|KL2|KL30|...,n492,1,2,1,0,None,None


In [20]:
df_prophages_2

,Prophage name,K-serotype monophyletic group,Id monophyletic group,Number of clades,Number of leafs,Number of new ancestors,Number of k-type swap,Nodes k-types,Nodes k-types all
0,GCF_017310325.1__phage26.fasta,KL15,n3842,1,2,1,0,None,None
1,GCF_017311795.1__phage12.fasta,KL111,n40554055,4,5,2,0,KL111,"KL111,KL111"
2,GCF_922827125.1__phage19.fasta,KL102,76.3/100,3,4,2,0,KL102,"KL102,KL102"
3,GCF_014840315.1__phage19.fasta,KL112,n10661066,3,4,1,0,None,None
4,GCF_003225795.1__phage33.fasta,KL30,n47034703,1,2,1,0,None,None
...,...,...,...,...,...,...,...,...,...
77798,GCF_905127585.1__phage2.fasta,KL64,GCF_905127585.1,0,1,0,0,None,None
77799,GCF_002187935.3__phage23.fasta,KL14,GCF_002187935.3,0,1,0,0,None,None
77800,GCF_017898175.1__phage24.fasta,KL64,GCF_017898175.1,0,1,0,0,None,None
77801,GCF_003955455.1__phage28.fasta,KL14,GCF_003955455.1,0,1,0,0,None,None


In [19]:
max(df_prophages_2["Number of new ancestors"])

72